In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import datetime

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

In [ ]:
url_train = 'https://raw.githubusercontent.com/xbadiam/Forecasting/refs/heads/main/data/output/train.csv'
train_df = pd.read_csv(url_train, index_col=0)
train_df.head(5)

In [ ]:
url_val = 'https://raw.githubusercontent.com/xbadiam/Forecasting/refs/heads/main/data/output/val.csv'
val_df = pd.read_csv(url_val, index_col=0)
val_df.head(5)

In [ ]:
url_test = 'https://raw.githubusercontent.com/xbadiam/Forecasting/refs/heads/main/data/output/test.csv'
test_df = pd.read_csv(url_test, index_col=0)
test_df.head(5)

In [ ]:
train_df = train_df.reset_index()
val_df   = val_df.reset_index()
test_df  = test_df.reset_index()

## Implementing a single-step linear model

We simply take all the inputs, assign a weight to each, take the sum, and generate a prediction. Remember that we are using the traffic volume as a target.

Unlike a baseline model, a linear model actually requires training. Thus, we'll define a compile_and_fit function that configures the model for training and then fits the model on the data, as shown in the following listing.

In [ ]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    """
    function that configures the model for training and then fits the model on the data, as 
    shown in the following listing.
   
    :param model: Model to be compiled and trained.
    :param window: An instance of the DataWindow class containing training and validation data.
    :param patience: Number of epochs with no improvement after which training will be stopped.
    :param max_epochs: sets amaximum number of epochs to train the model.
    """
    # Early stopping occurs if 3 consecutive epochs do not decrease 
    # the validation loss, as set by the patience parameter.
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')
    
    # The MSE is used as the loss function.
    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])
    
    # The model is fit using the training data from the window object.
    history = model.fit(window.train,
                        epochs=max_epochs,
                        validation_data=window.val,
                        callbacks=[early_stopping])
    
    return history
